In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal,norm
import math
from hmmlearn import hmm
import numpy.random as npr
from pyslds.models import DefaultSLDS

In [2]:
class SLDS(object):
    def __init__(self, Z = None, T=None, transition_dis = None, init_prob = None, transition_con = None, emission = None, Gamma = None, Sigma = None, u0 = None, V0 = None, y=None):

        if(transition_con is None or transition_dis is None or emission is None):
            raise ValueError("Set proper system dynamics.")

        self.y = y # T x N
        self.Z = Z # dimension of hidden discrete states
        self.M = transition_con.shape[1] # dimension of hidden continuous states
        self.T = T # number of observations
        self.N = self.y.shape[1] # dimension of the observations

        self.transition_dis = transition_dis # the discrete variable transition probability matrix, Z x Z
        self.init_prob = init_prob # the initial probability of the discrete variable, Z
        
        self.emission = emission # the emission probability of the continuous variable, Z x N x M
        self.transition_con = transition_con # the continuous variable transition probability matrix, Z x M x M 
        
        self.Gamma = Gamma # Gamma is the covariance matrix of noise term added to the hidden state transition, Z x M x M
        self.Sigma = np.eye(self.N) if Sigma is None else Sigma # Sigma is the covariance matrix of noise term added to the emission, N x N

        self.h = np.ones((self.T,self.Z)) # variational parameter h, T x Z
        self.q = np.ones((self.T,self.Z)) # variational parameter q, T x Z 
        
        self.P = np.zeros((self.T, self.Z, self.M, self.M))
        self.P[:,:,:,] = np.eye(self.M)  # P is an intermediate variable during inference, T x Z x M x M
        self.u = np.zeros((self.T, self.Z, self.M)) # T x Z x M x 1
        self.V = np.zeros((self.T, self.Z, self.M, self.M)) # T x Z x M x M
        self.K = np.zeros((self.T, self.Z, self.M, self.N)) # T x Z x M x N
        self.c = np.zeros((self.T, self.Z)) # T x Z x 1

        # for backward passing
        self.u_hat = np.zeros((self.T, self.Z, self.M)) # T x Z X M x 1
        self.V_hat = np.zeros((self.T, self.Z, self.M, self.M)) # T x Z x M x M
        self.J = np.zeros((self.T, self.Z, self.M, self.M)) # T x Z x M x M

        self.u0 = u0 # u0 is the initial estimate of the mean of z1, Z x M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of z1, Z x M x M
        
    def update_h(self):   
        print('updating h')     
        forward, backward = self.forward_backward(self.init_prob, self.q, self.transition_dis, self.y)

        self.forward = forward
        self.backward = backward
        a = np.zeros((self.T,self.Z))

        for i in range(self.T):
            for j in range(self.Z):
                a[i,j] = forward[i,j]*backward[i,j]
            temp = np.sum(a[i,:])
            a[i,:] = a[i,:]/temp

        self.h = a

    def forward_backward(self, init_prob, emission, transition, data):
        forward_hat = np.zeros((self.T,self.Z))
        backward_hat = np.zeros((self.T,self.Z))
        scale_factors = np.zeros((self.T))

        forward_hat[0,:] = init_prob * emission[0,:]
        scale_factors[0] = np.sum(forward_hat[0,:])
        forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
        
        for t in range(self.T-1):
            temp = np.matmul(forward_hat[t,:] ,transition) * emission[t+1,:] 
            scale_factors[t+1] = np.sum(temp)
            forward_hat[t+1,:] = temp/scale_factors[t+1]

        backward_hat[-1,:] = scale_factors[-1]
        for t in reversed(range(self.T-1)):
            temp = np.matmul(backward_hat[t+1,:] * emission[t+1,:],transition.T)
            backward_hat[t,:] = temp/scale_factors[t]
        return forward_hat, backward_hat

        # the end of forward-backward algorithm

    def update_q(self):
        print('updating q')
        self.Xt = np.zeros((self.T,self.Z,self.M))
        self.XtXt = np.zeros((self.T,self.Z,self.M,self.M))
        self.Xt = self.u_hat
        for i in range(self.T):
            for k in range(self.Z):
                self.XtXt[i,k] = self.V_hat[i,k] + np.outer(self.u_hat[i,k], self.u_hat[i,k].T)
        for i in range(self.T):
            for k in range(self.Z):

                Sigma_inverse = np.linalg.inv(self.Sigma)
                temp = -1/2 * self.y[i].T @ Sigma_inverse @ self.y[i] + self.y[i].T @ Sigma_inverse @ self.emission[k] @ self.Xt[i,k] - 1/2* np.trace(self.emission[k].T @ Sigma_inverse @ self.emission[k] @ self.XtXt[i,k])
                self.q[i,k] = np.exp(temp)
                # if self.q[i,k]==0:
                #     print(temp)
                #     print(-1/2 * self.y[i].T , Sigma_inverse , self.y[i])
                    

        # the end of update q

    def kalman_filtering(self):
        
        for k in range(self.Z):
            # print(self.emission.shape,self.u0)
            S_temp = np.matmul(np.matmul(self.emission[k], self.V0[k]), self.emission[k].T) + self.Sigma/self.h[0,k]
            Q_temp = np.matmul(self.emission[k], self.u0[k])
            I = np.eye(self.M)
            
            self.V[0,k] = np.matmul((I - np.matmul(np.matmul(np.matmul(self.V0[k], self.emission[k].T), np.linalg.inv(S_temp)), self.emission[k])), self.V0[k])
            self.P[0,k] = np.matmul(np.matmul(self.transition_con[k], self.V[0,k]), self.transition_con[k].T) + self.Gamma[k]
            self.K[0,k] = np.matmul(np.matmul(self.P[0,k], self.emission[k].T), np.linalg.inv(np.matmul(np.matmul(self.emission[k], self.P[0,k]), self.emission[k].T) + self.Sigma/self.h[0,k]))
            self.u[0,k] = self.u0[k] + np.matmul(self.K[0,k], self.y[0] - Q_temp)
            # print(self.y[0],Q_temp,S_temp)
            self.c[0,k] = multivariate_normal.pdf(self.y[0], Q_temp, S_temp)

            for i in range(1,self.T,1):
                I = np.eye(self.M)
                Q_temp = np.matmul(np.matmul(self.emission[k], self.transition_con[k]), self.u[i-1,k])
                
                self.V[i,k] = np.matmul((I - np.matmul(self.K[i-1,k], self.emission[k])), self.P[i-1,k])
                self.P[i,k] = np.matmul(np.matmul(self.transition_con[k], self.V[i,k]), self.transition_con[k].T) + self.Gamma[k]
                S_temp = np.matmul(np.matmul(self.emission[k], self.P[i,k]), self.emission[k].T) + self.Sigma/self.h[0,k]

                self.K[i,k] = np.matmul(np.matmul(self.P[i,k], self.emission[k].T), np.linalg.inv(S_temp))

                self.u[i,k] = np.matmul(self.transition_con[k], self.u[i-1,k]) + np.matmul(self.K[i-1,k], self.y[i] - Q_temp)

                self.c[i,k] = multivariate_normal.pdf(self.y[i], Q_temp, S_temp)

    def kalman_smoothing(self):
        for k in range(self.Z):
            self.u_hat[-1,k] = self.u[-1,k]
            self.V_hat[-1,k] = self.V[-1,k]

            for i in range(self.T-2,-1,-1):
                # print(self.V[i,k],self.transition_con[k].T,self.P[i,k])
                self.J[i,k] = np.matmul(np.matmul(self.V[i,k], self.transition_con[k].T), np.linalg.inv(self.P[i,k]))
                self.u_hat[i,k] = self.u[i,k] + np.matmul(self.J[i,k], self.u_hat[i+1,k] - np.matmul(self.transition_con[k], self.u[i,k]))
                self.V_hat[i,k] = self.V[i,k] + np.matmul(np.matmul(self.J[i,k], self.V_hat[i+1,k] - self.P[i,k]), self.J[i,k].T)
    
    def learning(self):
        Sigma = np.zeros((self.N,self.N))
        for k in range(self.Z):
            self.u0[k] = self.u_hat[0,k]
            self.V0[k] = self.V_hat[0,k] + np.outer(self.u_hat[0,k], self.u_hat[0,k].T) - np.outer(self.u_hat[0,k], self.u_hat[0,k].T)

            # E[z[n]] : M x 1
            # E[z[n]z[n-1].T] : M x M
            # E[z[n]z[n].T] : M x M
            a = np.zeros((self.T,self.Z))
            b = np.zeros((self.T,self.Z,self.Z))
            for i in range(self.T):
                for j in range(self.Z):
                    a[i,j] = self.forward[i,j] * self.backward[i,j]
                temp = np.sum(a[i,:])
                a[i,:] = a[i,:]/temp

            for t in range(self.T - 1):
                for i in range(self.Z):
                    for j in range(self.Z):
                        b[t,i,j] = self.forward[t,i] * self.backward[t+1,j] * self.transition_dis[i,j] * self.q[t+1,i]


            sub_1 = np.zeros((self.M,self.M))
            sub_2 = np.zeros((self.M,self.M))
            sub_3 = np.zeros((self.M,self.M))
            sub_4 = np.zeros((self.M,self.M))

            sub_5 = np.zeros((self.N,self.M))
            sub_6 = np.zeros((self.M,self.M))
            sub_7 = np.zeros((self.N,self.N))
            sub_8 = np.zeros((self.M,self.N))

            for i in range(1,self.T,1):
                sub_1 += np.matmul(self.V_hat[i,k],self.J[i-1,k].T) + np.outer(self.u_hat[i,k],self.u_hat[i-1,k].T) # z[n]z[n-1]
                
                sub_2 += self.V_hat[i-1,k] + np.outer(self.u_hat[i-1,k], self.u_hat[i-1,k].T)

                sub_3 += self.V_hat[i,k] + np.outer(self.u_hat[i,k], self.u_hat[i,k].T) # z[n]z[n]
                sub_4 += (np.matmul(self.V_hat[i,k],self.J[i-1,k].T) + np.outer(self.u_hat[i,k],self.u_hat[i-1,k].T)).T #z[n-1]z[n]

            for i in range(self.T):
                sub_5 += a[i,k] * np.outer(self.y[i], self.u_hat[i,k].T) # x[n] * E[z[n]].T
                sub_6 += a[i,k] * self.V_hat[i,k] + np.outer(self.u_hat[i,k], self.u_hat[i,k].T) # z[n]z[n]
                sub_7 += a[i,k] * np.outer(self.y[i], self.y[i].T) # x[n]x[n]
                sub_8 += a[i,k] * np.outer(self.u_hat[i,k], self.y[i].T) #E[z[n]] * x[n].T 
            

            self.transition_con[k] = np.matmul(sub_1, np.linalg.inv(sub_2))
            temp = 1/(self.T-1) * (sub_3 - np.matmul(self.transition_con[k],sub_4) - np.matmul(sub_1,self.transition_con[k].T) + np.matmul(np.matmul(self.transition_con[k],sub_2),self.transition_con[k].T))
            # self.Gamma[k] = 1/(self.T-1) * (sub_3 - np.matmul(self.transition_con[k],sub_4) - np.matmul(sub_1,self.transition_con[k].T) + np.matmul(np.matmul(self.transition_con[k],sub_2),self.transition_con[k].T))
            # print(self.Gamma[k].shape,temp.shape)
            # print(self.Gamma)

            self.emission[k] = np.matmul(sub_5, np.linalg.inv(sub_6))
            Sigma += 1/self.T * (sub_7 - np.matmul(self.emission[k],sub_8) - np.matmul(sub_5,self.emission[k].T) + 
                                 np.matmul(np.matmul(self.emission[k],sub_6),self.emission[k].T)) /self.h[0,k]
            self.Sigma = Sigma / self.Z



            for i in range(self.Z):
                for j in range(self.Z):
                    self.transition_dis[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

            for i in range(self.Z):
                self.init_prob[i] = a[0,i]/np.sum(a[0,:])
            print(f'transition_con is {self.transition_con}, and Gamma is {self.Gamma}')

            # for i in range(self.Z): # update_q will update the "emission probability" of the discrete variable
            #     for j in range(self.Z):
            #         self.emission[j,i] = np.sum(a[np.argwhere(self.y==i),j]) / np.sum(a[:,j])

In [3]:
def generate_examples(T=None, Z = None, M = None, N = None, transition_dis = None, init_prob = None, transition_con = None, emission = None, Gamma = None, Sigma = None, x0 = None):
 
    model = hmm.CategoricalHMM(n_components= Z)
    model.transmat_ = transition_dis
    model.startprob_ = init_prob
    model.emissionprob_ = np.array([[0.2,0.8],[0.9,0.1]])
    data,states = model.sample(n_samples = T,random_state=28)
    z = states
    x = np.zeros((T, M)) # for the continous variable
    y = np.zeros((T, N)) # the observation

    state = z[0]
    x[0] = x0[state]
    # if N == 1:
    #     y[0] = np.random.normal(np.matmul(emission[state], x[0]),Sigma)

    # else:
    y[0] = np.random.multivariate_normal(np.matmul(emission[state], x[0]),Sigma)
    
    for t in range(1,T,1):
        state = z[t]
        # if N ==1:
        #     x[t] = np.random.normal(np.matmul(transition_con[state],x[t-1]),Gamma[state])
        #     y[t] = np.random.normal(np.matmul(emission[state],x[t]),Sigma)
        # else:

        x[t] = np.random.multivariate_normal(np.matmul(transition_con[state],x[t-1]),Gamma[state])
        y[t] = np.random.multivariate_normal(np.matmul(emission[state],x[t]),Sigma)
    return z,x,y


In [4]:
def main():
	
	# n_dis = 2 # Z
	# n_con = 2 # M
	# n_obs = 2 # N
	# n_time = 400 # T

	# transition_dis = np.array([[0.95,0.05],[0.05,0.95]]) # the discrete variable transition probability matrix, Z x Z
	# init_prob = np.array([0.4,0.6]) # the initial probability of the discrete variable, N
	
	# emission = np.array([[[0.5, 0.5],[0.1, 0.9]],[[0.2, 0.8],[0.9, 0.1]]]) # the emission probability of the continuous variable, Z x N x M

	# transition_con = np.array([[[0.9, 0.1],[0.1, 0.9]],[[0.1, 0.9],[0.1, 0.9]]]) # the continuous variable transition probability matrix, Z x M x M 
	
	# Gamma = np.array([[[0.1, 0.3],[0.3, 0.1]],[[1, 0.5],[0.5, 1]]]) # Gamma is the covariance matrix of noise term added to the hidden state transition, Z x M x M
	# Sigma = np.array([[0.2, 0.8],[0.8, 0.2]]) # Sigma is the covariance matrix of noise term added to the emission, N x N
	# x0 = np.array([[0.2,0.2],[0.5, 0.5]]) # N x M



	# states_dis, states_con,obs = generate_examples(T=n_time, Z = n_dis, M = n_con, N = n_obs, transition_dis = transition_dis, init_prob = init_prob, 
	# 											transition_con = transition_con, emission = emission, Gamma = Gamma, Sigma = Sigma, x0 = x0)
	

	p_old = -10000
	tol = 0.01
	max_iter = 500
	
	n_dis = 2               # Number of discrete latent states
	n_obs = 2           # Observed data dimension
	n_con = 2        # Latent state dimension
	D_input = 0         # Exogenous input dimension
	n_time = 2000            # Number of time steps to simulate

	true_model = DefaultSLDS(n_dis, n_obs, n_con, D_input)
	inputs = npr.randn(n_time, D_input)
	obs, states_con, states_dis = true_model.generate(n_time, inputs=inputs)
	# print(obs.shape,states_con.shape,states_dis.shape)

	transition_dis_init = np.array([[0.7,0.3],[0.3,0.7]]) 
	init_prob_init = np.array([0.9,0.1]) 
	
	emission_init = np.array([[[0.3, 0.7],[0.2, 0.8]],[[0.25, 0.75],[0.4, 0.6]]]) 
	transition_con_init = np.array([[[0.9, 0.1],[0.1, 0.9]],[[0.1, 0.9],[0.1, 0.9]]])
	
	Gamma_init = np.array([[[1.0, 0],[0, 1.0]],[[1.0, 0],[0, 1.0]]])  
	Sigma_init = np.array([[1.0, 0],[0, 1.0]])
	u0 = np.array([[0.3, 0.3],[0.5, 0.5]]) # Z x M x 1
	V0 = Gamma_init # Z x M x M

	slds = SLDS(Z = n_dis, T=n_time, transition_dis = transition_dis_init, init_prob = init_prob_init, transition_con = transition_con_init, 
			 emission = emission_init, Gamma = Gamma_init, Sigma = Sigma_init, u0 = u0, V0 = V0, y=obs)
	
	for ite in range(max_iter):
		# print(obs)
		slds.update_q()
		slds.update_h()
	
		# print(slds.forward,slds.backward)
		# print(slds.q,slds.h)
		slds.kalman_filtering()
		slds.kalman_smoothing()
		slds.learning()

		p = np.sum(np.log(slds.c))
		print(f'The current iteration is: {ite}. The likelihood is {p}',end='\r')
		if p>p_old and p - p_old < tol:
			break
		p_old = p

	print('u0\n',slds.u0,'\nV0\n',slds.V0,'\ntransition_dis\n',slds.transition_dis,'\ntransition_con\n',slds.transition_con,'\nemission\n',slds.emission,'\nGamma\n',
	   slds.Gamma,'\ninit_prob\n',slds.init_prob,'\nGamma\n',slds.Gamma,'\nSigma\n',slds.Sigma)
	return slds,states_dis,states_con,obs

slds,states_dis,states_con,obs = main()

updating q
updating h
transition_con is [[[ 0.78114884 -0.086714  ]
  [-0.1307298   0.38477231]]

 [[ 0.1         0.9       ]
  [ 0.1         0.9       ]]], and Gamma is [[[ 0.88835137 -0.19662979]
  [-0.19662979  0.44274179]]

 [[ 1.          0.        ]
  [ 0.          1.        ]]]
transition_con is [[[ 0.78114884 -0.086714  ]
  [-0.1307298   0.38477231]]

 [[ 0.09230375  0.83083988]
  [ 0.06388302  0.57533888]]], and Gamma is [[[ 0.88835137 -0.19662979]
  [-0.19662979  0.44274179]]

 [[ 0.97850646 -0.05558596]
  [-0.05558596  0.69395275]]]
updating qt iteration is: 0. The likelihood is -13671.840675720148
updating h
transition_con is [[[ 0.78086008 -0.08707234]
  [-0.1312196   0.38330752]]

 [[ 0.09230375  0.83083988]
  [ 0.06388302  0.57533888]]], and Gamma is [[[ 0.88795697 -0.1971149 ]
  [-0.1971149   0.44057413]]

 [[ 0.97850646 -0.05558596]
  [-0.05558596  0.69395275]]]
transition_con is [[[ 0.78086008 -0.08707234]
  [-0.1312196   0.38330752]]

 [[ 0.09230296  0.83083358]
  [ 